In [10]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Initialize Firestore
cred = credentials.Certificate(r'C:\Users\vishw\Downloads\movielens-445823-3a8bec19f03d.json')  # Replace with your service account file
firebase_admin.initialize_app(cred)

# Access Firestore
db = firestore.client()

# Retrieve a document
def get_movie_data(movie_id):
    movie_ref = db.collection('movies').document(str(movie_id))
    movie = movie_ref.get()

    if movie.exists:
        print(f"Movie data: {movie.to_dict()}")
    else:
        print("No such document!")

# Example: Get data for movie with movie_id=1
get_movie_data(1)


Movie data: {'tmdb': '862', 'year': '1995', 'avg_rating': 3.9209302325581397, 'total_rating': 843.0, 'imdb': '0114709', 'poster': 'http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', 'title': 'Toy Story', 'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], 'ratings': 215}


In [14]:


# Retrieve all documents from the 'movies' collection
def get_all_movies():
    movies_ref = db.collection('movies')  # Reference to the 'movies' collection
    docs = movies_ref.stream()  # Stream all documents from the collection

    for doc in docs:
        print(f"Movie ID: {doc.id}, Data: {doc.to_dict()}")

# Call the function to retrieve all movies
get_all_movies()


Movie ID: 1, Data: {'tmdb': '862', 'year': '1995', 'avg_rating': 3.9209302325581397, 'total_rating': 843.0, 'imdb': '0114709', 'poster': 'http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', 'title': 'Toy Story', 'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], 'ratings': 215}
Movie ID: 10, Data: {'tmdb': '710', 'year': '1995', 'avg_rating': 3.496212121212121, 'total_rating': 461.5, 'imdb': '0113189', 'poster': 'http://image.tmdb.org/t/p/w154/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg', 'title': 'GoldenEye', 'genres': ['Action', 'Adventure', 'Thriller'], 'ratings': 132}
Movie ID: 100, Data: {'tmdb': '11062', 'year': '1996', 'avg_rating': 2.7857142857142856, 'total_rating': 39.0, 'imdb': '0115907', 'poster': 'http://image.tmdb.org/t/p/w154/ttdjD9NpxXsTJidPT3o1Gk0oTkG.jpg', 'title': 'City Hall', 'genres': ['Drama', 'Thriller'], 'ratings': 14}
Movie ID: 100044, Data: {'tmdb': '69372', 'year': '2011', 'avg_rating': 4.0, 'total_rating': 4.0, 'imdb': '1806234', 'title'

In [17]:
downloads_folder = r"C:\Users\vishw\Downloads\movies_data.csv"

def save_movies_to_csv():
    movies_ref = db.collection('movies')  # Reference to the 'movies' collection
    docs = movies_ref.stream()  # Stream all documents from the collection

    # Open a CSV file to write the data in the Downloads folder
    with open(downloads_folder, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Movie ID', 'tmdb', 'year', 'avg_rating', 'total_rating', 'imdb', 'poster', 'title', 'genres', 'ratings'])
        
        writer.writeheader()  # Write the header row
        
        for doc in docs:
            movie_data = doc.to_dict()
            # Write the movie data into the CSV file
            movie_data['Movie ID'] = doc.id  # Add Movie ID from Firestore document ID
            writer.writerow(movie_data)

    print(f"Data has been written to '{downloads_folder}'.")

# Call the function to save movies to CSV
save_movies_to_csv()

Data has been written to 'C:\Users\vishw\Downloads\movies_data.csv'.


In [19]:
import pandas as pd

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\vishw\Downloads\movies_data.csv')

# Convert the 'genres' column from string representation of lists to actual lists
df['genres'] = df['genres'].apply(eval)

# Explode the 'genres' column so that each genre has its own row
df_exploded = df.explode('genres')

# Now you have multiple rows for each genre
# You can save this into a new CSV or work with it
df_exploded.to_csv(r'C:\Users\vishw\Downloads\movies_data_exploded.csv', index=False)

# Display the first few rows of the new DataFrame
print(df_exploded.head())

   Movie ID   tmdb  year  avg_rating  total_rating    imdb  \
0         1  862.0  1995     3.92093         843.0  114709   
0         1  862.0  1995     3.92093         843.0  114709   
0         1  862.0  1995     3.92093         843.0  114709   
0         1  862.0  1995     3.92093         843.0  114709   
0         1  862.0  1995     3.92093         843.0  114709   

                                              poster      title     genres  \
0  http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4ve...  Toy Story  Adventure   
0  http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4ve...  Toy Story  Animation   
0  http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4ve...  Toy Story   Children   
0  http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4ve...  Toy Story     Comedy   
0  http://image.tmdb.org/t/p/w154/rhIRbceoE9lR4ve...  Toy Story    Fantasy   

   ratings  
0    215.0  
0    215.0  
0    215.0  
0    215.0  
0    215.0  


In [23]:
import pandas as pd

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\vishw\Downloads\movies_data_exploded.csv')

# Filter the rows where 'Animation' is one of the genres
df_animation = df[df['genres'] == 'Animation']

In [ ]:
import pandas as pd

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\vishw\Downloads\movies_data_exploded.csv')

# Filter the rows where 'Animation' is one of the genres
df_animation = df[df['genres'] == 'Animation']

# Group by 'genres' and calculate the sum of 'total_rating' and 'ratings'
df_sum = df_animation.groupby('genres').agg(
    total_rating_sum=('total_rating', 'sum'),
    ratings_sum=('ratings', 'sum')
).reset_index()


# Display the result
print(df_sum)

      genres  total_rating_sum  ratings_sum
0  Animation           25366.0       6988.0


In [8]:
import pandas as pd

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\vishw\Downloads\movies_data_exploded.csv')

# Split the 'genres' column into lists (if not already a list) and explode into individual rows
df['genres'] = df['genres'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))

# Explode the list of genres into individual rows
df_exploded = df.explode('genres')

# Get unique genres
unique_genres = df_exploded['genres'].unique()

# Initialize an empty DataFrame to store the results
df_result = pd.DataFrame()

# Loop through each unique genre
for i in unique_genres:
    # Filter the rows where the genre is one of the unique genres
    df_genre = df_exploded[df_exploded['genres'] == i]
    
    # Group by 'genres' and calculate the sum of 'total_rating' and 'ratings'
    df_sum = df_genre.groupby('genres').agg(
        total_rating_sum=('total_rating', 'sum'),
        ratings_sum=('ratings', 'sum')
    ).reset_index()

    df_sum['average_rating'] = df_sum['total_rating_sum'] / df_sum['ratings_sum']
    
    # Concatenate the result to df_result
    df_result = pd.concat([df_result, df_sum], ignore_index=True)

# Display the final result
print(df_result)


                genres  total_rating_sum  ratings_sum  average_rating
0            Adventure           84771.5      24161.0        3.508609
1            Animation           25366.0       6988.0        3.629937
2             Children           31426.5       9208.0        3.412956
3               Comedy          132183.5      39053.0        3.384721
4              Fantasy           41312.5      11834.0        3.491001
5               Action          105629.0      30635.0        3.447984
6             Thriller           92415.5      26452.0        3.493706
7                Drama          153296.5      41928.0        3.656184
8          Documentary            4629.5       1219.0        3.797785
9               Horror           23755.5       7291.0        3.258195
10                IMAX           14998.0       4145.0        3.618335
11             Romance           63552.0      18124.0        3.506511
12              Sci-Fi           59587.0      17243.0        3.455721
13               Cri

In [9]:
df_result_sorted = df_result.sort_values(by='average_rating', ascending=False)


In [11]:
df_result_sorted.head(3)

,genres,total_rating_sum,ratings_sum,average_rating
18,Film-Noir,3410.5,870.0,3.920115
17,War,18504.5,4859.0,3.808294
8,Documentary,4629.5,1219.0,3.797785
